In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import glob
import pickle
import re
from difflib import SequenceMatcher
from dateutil import parser

In [46]:
covers = pickle.load(open("data/covers_algo_results.p", "rb"))

In order to manipulate dates, we import the parser from the dateutil python package, and convert the date column in string. Some date containing only the year are floats so they are converted to a string of the form "(year).0", thus we also need to remove all the ".0"

In [47]:
def multiple_originals(covers, col):
    groupbyclique = covers.groupby('clique_id')

    multi_originals = []
    for c, group in groupbyclique:
        nb_original = len(group[group[col] == "original"])
        if nb_original > 1:
            multi_originals.append(c)
    return multi_originals

In [71]:
dup_originals = multiple_originals(covers, 'second_algo_result')

In [72]:
len(dup_originals)

0

In [50]:
covers[covers['clique_id'] == 6]

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank,first_algo_result,second_algo_result
14,TREVWUZ128F4263A9B,AR9UYPT1187B9AE833,45769,6,Hear'Say,Monday Monday,Hear'Say,English,2001,9133,9133,United Kingdom,NaN,2001,1,NaN,nan
15,TRGYREY128E0791913,ARQ294N1187FB53D2A,9133,6,The Mamas & The Papas,Monday_ Monday,The Mamas & The Papas,Unavailable,NaN,0,9133,United States,NaN,nan,2,original,nan


In [7]:
def get_val(covers, track, col):
    index = covers[covers['trackID'] == track].index[0]
    return covers.get_value(index, col)

In [8]:
covers['date'] = covers['date'].map(lambda x : str(x).replace(".0", "")) 

covers['country'].replace("Missing", np.nan, inplace=True)
covers['country'].replace("Unavailable", np.nan, inplace=True)

covers["final_original"] = covers['trackID'].\
map(lambda x : "nan" if get_val(covers, x, 'clique_id') in dup_originals else get_val(covers, x, 'second_algo_result'))

covers.head(15)

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank,first_algo_result,second_algo_result,final_original
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977,1,original,original,original
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660,United Kingdom,Pop,2009,2,NaN,nan,nan
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066,United States,Alternative,2002,1,original,original,original
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066,United States,Alternative,2003,2,NaN,nan,nan
10,TRQNZCE128E078A9C0,ARWILYB1187FB37DFE,52011,4,Bananarama,More_ More_ More,Bananarama,English,"March 20, 1993",52010,52010,United Kingdom,Pop,1993,2,NaN,nan,nan
11,TRMBSQR128F92DF66E,ARPI2DX1187FB4CED4,52010,4,Andrea True Connection,More More More,Andrea True Connection,English,1976,52010,52010,United States,R&B/Soul,1976,1,original,original,original
14,TREVWUZ128F4263A9B,AR9UYPT1187B9AE833,45769,6,Hear'Say,Monday Monday,Hear'Say,English,2001,9133,9133,United Kingdom,NaN,2001,1,NaN,nan,nan
15,TRGYREY128E0791913,ARQ294N1187FB53D2A,9133,6,The Mamas & The Papas,Monday_ Monday,The Mamas & The Papas,Unavailable,nan,0,9133,United States,NaN,nan,2,original,nan,nan
16,TRBLBSR128F425EBFE,AR36FFP1187B9926D7,59663,7,Silicon Teens,Memphis Tennessee,Silicon Teens,English,September 1980,575,575,United Kingdom,Electronic,1980,5,NaN,nan,nan
17,TRLRWJK128F427D602,AR6NBDC1187FB4D96D,575,7,Chuck Berry,Memphis_ Tennessee,Chuck Berry,English,May 1959,575,575,United States,Rock,1959,1,original,original,original


Now, we define the merge_algo function that, given a dataframe of tracks within a clique will return the original. The original returned is the one with the oldest date of release. If the oldest year appears mutliple times, we will check if we can find the original from the first algorithm. By doing some tests, there are only 73 cliques for which we cannot determine the original by merging the two algorithms. We will then manually define the original for these cliques.

In [15]:
def find_original(tracks):
    default = parser.parse("December 31, 2017")
    dates = tracks['date'].map(lambda x : parser.parse(str(x), default=default)).tolist()
    oldest = dates.index(min(dates))
    count = dates.count(min(dates))
    index = tracks.iloc[[oldest]].index[0]
    clique_id = tracks.get_value(index, 'clique_id')
    if count > 1:
        date = tracks.get_value(index, "date")
        tracks_date = tracks[tracks['date'] == date]
        first_algo = tracks[tracks['first_algo_result'] == "original"]
        if first_algo.empty:
            null_nb = tracks_date.isnull().sum(axis=1).tolist()
            more_complete = null_nb.index(min(null_nb))
            index = tracks.iloc[[more_complete]].index[0]
        else:
            index = first_algo.head(1).index[0]
    return index
    

In [16]:
def keep_one_original(covers, clique_id):
    clique = covers[covers['clique_id'] == clique_id]
    tracks = clique[clique['second_algo_result'] == 'original']
    original_index = find_original(tracks)
    covers.set_value(original_index, 'final_original', "original")

In [17]:
for c in dup_originals:
    keep_one_original(covers, c)

In [20]:
covers['final_original'].replace('nan', 'cover', inplace=True)

In [21]:
covers

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank,first_algo_result,second_algo_result,final_original
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977,1,original,original,original
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660,United Kingdom,Pop,2009,2,NaN,nan,cover
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066,United States,Alternative,2002,1,original,original,original
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066,United States,Alternative,2003,2,NaN,nan,cover
10,TRQNZCE128E078A9C0,ARWILYB1187FB37DFE,52011,4,Bananarama,More_ More_ More,Bananarama,English,"March 20, 1993",52010,52010,United Kingdom,Pop,1993,2,NaN,nan,cover
11,TRMBSQR128F92DF66E,ARPI2DX1187FB4CED4,52010,4,Andrea True Connection,More More More,Andrea True Connection,English,1976,52010,52010,United States,R&B/Soul,1976,1,original,original,original
14,TREVWUZ128F4263A9B,AR9UYPT1187B9AE833,45769,6,Hear'Say,Monday Monday,Hear'Say,English,2001,9133,9133,United Kingdom,NaN,2001,1,NaN,nan,cover
15,TRGYREY128E0791913,ARQ294N1187FB53D2A,9133,6,The Mamas & The Papas,Monday_ Monday,The Mamas & The Papas,Unavailable,nan,0,9133,United States,NaN,nan,2,original,nan,cover
16,TRBLBSR128F425EBFE,AR36FFP1187B9926D7,59663,7,Silicon Teens,Memphis Tennessee,Silicon Teens,English,September 1980,575,575,United Kingdom,Electronic,1980,5,NaN,nan,cover
17,TRLRWJK128F427D602,AR6NBDC1187FB4D96D,575,7,Chuck Berry,Memphis_ Tennessee,Chuck Berry,English,May 1959,575,575,United States,Rock,1959,1,original,original,original


In [24]:
check_dup = multiple_originals(covers, 'final_original')
len(check_dup)

0

In [25]:
pickle.dump(covers, open("data/covers_merge_algo.p", "wb"))

In [76]:
cover_songs = covers_merge[covers_merge['final_original'] != "original"]

In [88]:
countcovers = cover_songs.groupby('year').count()[['trackID']]
countcovers.columns = [['count']]

In [102]:
countcovers.reset_index(inplace = True)

In [272]:
countcovers[countcovers['year'].isin(["2016","2017"])]

,year,count


In [285]:
def print_values(y1):
    i1 = countoriginals[countoriginals['year'] == str(y1)].index[0]
    v1 = countoriginals.get_value(i1, 'count')
    y2 = y1+1
    #v1 = 0
    i2 = countoriginals[countoriginals['year'] == str(y2)].index[0]
    v2 = countoriginals.get_value(i2, 'count')
    height = v1+v2
    
    y = 329 - height
    print("height = " + str(height) + "\ny = " + str(y))

In [332]:
print_values(2008)

height = 23
y = 306


In [283]:
originals = covers_merge[covers_merge['final_original'] == "original"]
countoriginals = originals.groupby('year').count()[['trackID']]
countoriginals.columns = [['count']]
countoriginals.reset_index(inplace=True)

In [284]:
countoriginals

,year,count
0,,17
1,1911,1
2,1923,1
3,1925,4
4,1926,2
5,1927,8
6,1928,13
7,1929,14
8,1930,11
9,1931,14
